    francesco urbani
    Thu Nov 11 16:05:04 CET 2021

Auxiliary functions here

In [33]:
def print_bin(x, n=8):
    ans = format(x, "b").zfill(n)
    ans = "_".join(ans[i : i + 4] for i in range(0, len(ans), 4))
    return ans

---

In [34]:
def sign_reg_ui(bits):
    """
    returns the sign of the regime,
    determined peeking at 1st bit after sign
    """
    return (bits & 0x40) != 0

In [35]:
def separate_bits_tmp(bits):
    k = 0
    tmp = bits << 2
    if sign_reg_ui(bits):
        while (tmp & 0x80) != 0:
            k += 1
            tmp = tmp << 1
    else:
        k = -1
        while (tmp & 0x80) == 0:
            k -= 1
            tmp = tmp << 1
        tmp &= 0x7F
    return (k, tmp & 0xFF)


separate_bits_tmp(10)

(-3, 32)

In [36]:
def separate_bits(bits):
    k, tmp = separate_bits_tmp(bits)
    return (k, tmp | 0x80)


separate_bits(10)

(-3, 160)

In [37]:
def checked_shr(bits, rhs):  # -> (u8)
    # https://doc.rust-lang.org/std/primitive.u32.html#method.checked_shr
    return bits >> rhs


checked_shr(0x40, 4)

4

In [38]:
def calculate_regime(k):
    if k < 0:
        length = -k & 0xFFFF_FFFF
        return (checked_shr(0x40, length), False, length)
    else:
        length = (k + 1) & 0xFFFF_FFFF
        return (0x7F - checked_shr(0x7F, length), True, length)


calculate_regime(-2)

(16, False, 2)

In [39]:
def pack_to_ui(regime, frac):
    return regime + frac

In [40]:
def calc_ui(k, frac16):
    regime, reg_s, reg_len = calculate_regime(k)
    print(f"{regime=}, {reg_s=}, {reg_len=}")

    if reg_len > 6:
        if reg_s:
            return 0x7F
        else:
            return 0x01
    else:
        frac16 = (frac16 & 0x3FFF) >> reg_len
        frac = (frac16 >> 8) & 0xFF
        #                    ^^^^^^ => as u8
        bit_n_plus_one = (frac16 & 0x80) != 0
        u_z = pack_to_ui(regime, frac)
        if bit_n_plus_one:
            bits_more = (frac16 & 0x7F) != 0
            u_z += (u_z & 1) | (bits_more & 0xFF)
        return u_z & 0xFF


calc_ui(4, 0b1010100000000000)

regime=124, reg_s=True, reg_len=5


125

In [41]:
def from_bits(bits, sign):
    if sign:
        return c2(bits)
    else:
        return bits

In [42]:
def c2(a):
    return (~a + 1) & 0xFF


def wrapping_neg(a):
    if (a & 0x80) == 1:  # is neg
        return a
    else:
        return c2(a)


n = -100
print(get_bin(n))

get_bin(wrapping_neg(n & 0xFF))

-1100100


'01100100'

In [43]:
def is_nar(a):
    return (a ^ 0x80) == 0


def is_zero(a):
    return a == 0

In [70]:
def p8mul(a, b):

    ui_a, ui_b = a, b

    if is_nar(a) or is_nar(b):
        return 0x80  # NaR
    elif is_zero(a) or is_zero(b):
        return 0  # zero

    sign_a = ui_a & 0x80
    sign_b = ui_b & 0x80
    sign_z = sign_a ^ sign_b

    if sign_a:
        ui_a = wrapping_neg(ui_a)
        print(ui_a)
    if sign_b:
        ui_b = wrapping_neg(ui_b)
        print(ui_b)

    k_a, frac_a = separate_bits(ui_a)
    k_b, frac_b = separate_bits(ui_b)

    print(f"{k_a=}, {bin(frac_a)=}")
    print(f"{k_b=}, {bin(frac_b)=}")

    k_c = k_a + k_b

    frac16 = (frac_a * frac_b) & 0xFFFF

    rcarry = (frac16 & 0x8000) != 0
    print(f"{rcarry=}")

    if rcarry:
        k_c += 1
        frac16 = frac16 >> 1

    print(f"{k_c=}")
    print(f"{bin(frac16)=}")

    u_z = calc_ui(k_c, frac16)
    print(f"{bin(u_z)=}")

    return from_bits(u_z, sign_z)

---

In [71]:
import softposit as sp

---
## example 1

In [72]:
p8mul(0b00000000, 0b01011000)

0

In [73]:
print_bin(_)

'0000_0000'

In [74]:
sp.posit8(bits=p8mul(0b11110110, 0b10101001)).toBinaryFormatted()

10
87
k_a=-3, bin(frac_a)='0b10100000'
k_b=0, bin(frac_b)='0b11011100'
rcarry=True
k_c=-2
bin(frac16)='0b100010011000000'
regime=16, reg_s=False, reg_len=2
bin(u_z)='0b10001'
00010001


--- 
## example 2 

In [75]:
a1 = sp.posit8(12)
b1 = sp.posit8(1.75)

In [76]:
a1.toBinaryFormatted()
b1.toBinaryFormatted()

01111010
01011000


In [77]:
c1 = a1 * b1
c1

24.0

In [79]:
sp.posit8(bits=p8mul(0b01111010, 0b01011000))

k_a=3, bin(frac_a)='0b11000000'
k_b=0, bin(frac_b)='0b11100000'
rcarry=True
k_c=4
bin(frac16)='0b101010000000000'
regime=124, reg_s=True, reg_len=5
bin(u_z)='0b1111101'


24.0

In [80]:
c1.toBinaryFormatted()

01111101


In [81]:
# example 2

In [82]:
p8mul(0b01000111, 0b10001111)

113
k_a=0, bin(frac_a)='0b10011100'
k_b=2, bin(frac_b)='0b10010000'
rcarry=False
k_c=2
bin(frac16)='0b101011111000000'
regime=112, reg_s=True, reg_len=3
bin(u_z)='0b1110011'


141

In [83]:
sp.posit8(bits=0b01000111), sp.posit8(bits=0b10001111)

(1.21875, -4.5)

In [69]:
p8mul(0x31, 0x02)

k_a=-1, bin(frac_a)='0b11000100'
k_b=-5, bin(frac_b)='0b10000000'
bin(frac16)='0b110001000000000'
rcarry=False
k_c=-6
regime=1, reg_s=False, reg_len=6
bin(u_z)='0b10'


2

In [59]:
p8mul(0b00000000, 0b01100000)

0

In [60]:
bin(_)

'0b0'

In [61]:
sp.posit8(0.0) * sp.posit8(2)

0.0

In [62]:
_.toBinaryFormatted()

00000000


In [63]:
sp.posit8(2).toBinaryFormatted()

01100000
